In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [3]:
#spacy for lemmatization
import spacy

In [4]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/Users/root1/Documents/lda/myvenv/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['tagum', 'city', 'ordinance', 'municipal', 'municipality', 'thereof'])

In [7]:
# Import Dataset
df = pd.read_csv('/Users/root1/Documents/lda/tagum_ordinances.csv', header = 0)
print(df)
df.head()

                                     OR  \
0        CITY ORDINANCE No. 842, s-2018   
1        CITY ORDINANCE NO. 825, s-2017   
2        CITY ORDINANCE NO. 837, s-2018   
3        CITY ORDINANCE NO. 844, s-2018   
4        CITY ORDINANCE NO. 838, s-2018   
..                                  ...   
670  MUNICIPAL ORDINANCE NO. 05, s-1974   
671  MUNICIPAL ORDINANCE NO. 04, s-1974   
672  MUNICIPAL ORDINANCE NO. 01, s-1974   
673    MUNICIPAL ORDINANCE NO. 03, 1973   
674  MUNICIPAL ORDINANCE NO. 02, S-1973   

                                                    OT  
0    “AN ORDINANCE REGULATING THE PRACTICE OF TRADI...  
1    “AN ORDINANCE RECONSTITUTING THE MEMBERS OF TH...  
2    “AN ORDINANCE ESTABLISHING FUNERAL AND BURIAL ...  
3    AN ORDINANCE AMENDING THE TITLE, SECTION 1, SE...  
4    “AN ORDINANCE CREATING THE CITY HISTORICAL, CU...  
..                                                 ...  
670  “AN ORDINANCE AMENDING SECTION 10 OF MUNICIPAL...  
671  “AN ORDINANCE REGULATI

,OR,OT
0,"CITY ORDINANCE No. 842, s-2018",“AN ORDINANCE REGULATING THE PRACTICE OF TRADI...
1,"CITY ORDINANCE NO. 825, s-2017",“AN ORDINANCE RECONSTITUTING THE MEMBERS OF TH...
2,"CITY ORDINANCE NO. 837, s-2018",“AN ORDINANCE ESTABLISHING FUNERAL AND BURIAL ...
3,"CITY ORDINANCE NO. 844, s-2018","AN ORDINANCE AMENDING THE TITLE, SECTION 1, SE..."
4,"CITY ORDINANCE NO. 838, s-2018","“AN ORDINANCE CREATING THE CITY HISTORICAL, CU..."


In [8]:
# Convert to list
data = list(df.OT)
print(data[:1])

['“AN ORDINANCE REGULATING THE PRACTICE OF TRADITIONAL HOME BIRTH DELIVERY OR DELIVERIES ATTENDED BY A TRADITIONAL BIRTH ATTENDANT OR MANANABANG”.']


In [9]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub("\"", "", sent) for sent in data]

# Remove Emails
data = [re.sub('\S*ING\S*\s?', '', sent) for sent in data]


print(data[:5])

['“AN ORDINANCE THE PRACTICE OF TRADITIONAL HOME BIRTH DELIVERY OR DELIVERIES ATTENDED BY A TRADITIONAL BIRTH ATTENDANT OR MANANABANG”.', '“AN ORDINANCE THE MEMBERS OF THE CITY TRICYCLE AND REGULATORY BOARD (CTFRB) ITS FUNCTIONS AND PROCEDURES AND MOTORIZED TRICYCLE FOR HIRE (MTH), TRICYCLE UTILITY VEHICLE (TUV) AND TRICYCLE UTILITY FOR HIRE (TUH) GUIDELINES, REGULATIONS AND OPERATIONS, THE COLLECTION OF FEES, CHARGES,ADMINISTRATIVE PROVISIONS AND PENALTIES FOR VIOLATION THEREOF”.', '“AN ORDINANCE FUNERAL AND BURIAL ASSISTANCE FOR INDIGENT TAGUMENYOS, ITS MANAGEMENT, PROCEDURES AND GUIDELINES AND FUNDS THEREFOR”.', 'AN ORDINANCE THE TITLE, SECTION 1, SECTION 2 AND SECTION 3 OF CITY ORDINANCE NO. 820, S-2017 “AN ORDINANCE INTO A LOAN AGREEMENT WITH THE DEVELOPMENT BANK OF THE PHILIPPINES, IN THE AMOUNT OF SIX HUNDRED EIGHTY MILLION PESOS (P 680,000,000.00) TO FUND THE ACQUISITION OF HEAVY EQUIPMENT,ACQUISITION OF SERVICE VEHICLES, ACQUISITION OF continue reading : CITY ORDINANCE NO. 844

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:5])

[['an', 'ordinance', 'the', 'practice', 'of', 'traditional', 'home', 'birth', 'delivery', 'or', 'deliveries', 'attended', 'by', 'traditional', 'birth', 'attendant', 'or', 'mananabang'], ['an', 'ordinance', 'the', 'members', 'of', 'the', 'city', 'tricycle', 'and', 'regulatory', 'board', 'ctfrb', 'its', 'functions', 'and', 'procedures', 'and', 'motorized', 'tricycle', 'for', 'hire', 'mth', 'tricycle', 'utility', 'vehicle', 'tuv', 'and', 'tricycle', 'utility', 'for', 'hire', 'tuh', 'guidelines', 'regulations', 'and', 'operations', 'the', 'collection', 'of', 'fees', 'charges', 'administrative', 'provisions', 'and', 'penalties', 'for', 'violation', 'thereof'], ['an', 'ordinance', 'funeral', 'and', 'burial', 'assistance', 'for', 'indigent', 'tagumenyos', 'its', 'management', 'procedures', 'and', 'guidelines', 'and', 'funds', 'therefor'], ['an', 'ordinance', 'the', 'title', 'section', 'section', 'and', 'section', 'of', 'city', 'ordinance', 'no', 'an', 'ordinance', 'into', 'loan', 'agreement',

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['an', 'ordinance', 'the', 'practice', 'of', 'traditional', 'home', 'birth', 'delivery', 'or', 'deliveries', 'attended', 'by', 'traditional', 'birth', 'attendant', 'or', 'mananabang']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:4])

[['practice', 'traditional', 'home', 'birth', 'delivery', 'delivery', 'attend', 'traditional', 'birth', 'attendant', 'mananabang'], ['function', 'procedure', 'tricycle', 'utility', 'vehicle', 'tricycle', 'utility', 'hire', 'guideline', 'operation', 'collection', 'fee', 'charge', 'administrative', 'provision', 'penalty', 'violation'], ['indigent', 'management', 'procedure', 'guideline', 'fund'], ['section', 'loan', 'amount', 'acquisition', 'equipment', 'acquisition', 'service', 'vehicle', 'acquisition', 'continue_reading']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 2)]]


In [15]:
id2word[0]

'attend'

In [16]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('attend', 1),
  ('attendant', 1),
  ('birth', 2),
  ('delivery', 2),
  ('home', 1),
  ('mananabang', 1),
  ('practice', 1),
  ('traditional', 2)]]

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.161*"section" + 0.055*"road" + 0.054*"tax" + 0.032*"implementation" + 0.024*"annual" + 0.023*"material" + 0.023*"code" + 0.021*"article" + 0.021*"bus" + 0.020*"jeepney"'), (1, '0.069*"service" + 0.042*"private" + 0.039*"development" + 0.037*"mayor" + 0.035*"management" + 0.029*"appropriation" + 0.024*"temporary" + 0.023*"issuance" + 0.023*"official" + 0.021*"clearance"'), (2, '0.064*"situate" + 0.041*"system" + 0.036*"supply" + 0.032*"secure" + 0.028*"driver" + 0.028*"disposal" + 0.026*"civil" + 0.025*"set" + 0.021*"highway" + 0.021*"facility"'), (3, '0.082*"market" + 0.044*"public" + 0.041*"street" + 0.039*"amount" + 0.032*"new" + 0.027*"portion" + 0.023*"rule" + 0.023*"medical" + 0.021*"benefit" + 0.018*"regulation"'), (4, '0.074*"establishment" + 0.057*"operation" + 0.050*"regulation" + 0.041*"revise" + 0.035*"student" + 0.034*"park" + 0.032*"sale" + 0.031*"commercial" + 0.027*"lot" + 0.023*"close"'), (5, '0.092*"penalty" + 0.087*"violation" + 0.048*"public" + 0.034*"vehicle

In [19]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.82037510426432

Coherence Score:  0.49747051572831386


In [20]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.284675  0.159838       1        1  18.848725
3     -0.056880 -0.249604       2        1  10.892713
11     0.219499  0.144491       3        1  10.096346
9     -0.000171 -0.110515       4        1   8.681745
6      0.070955 -0.024482       5        1   8.159320
0      0.031208  0.011294       6        1   8.004615
7     -0.131118  0.083447       7        1   7.123207
4     -0.098959  0.012046       8        1   6.785554
10     0.154377  0.111017       9        1   6.750020
8      0.023039 -0.072572      10        1   6.299204
1      0.065596 -0.032761      11        1   5.793656
2      0.007130 -0.032200      12        1   2.564895, topic_info=          Term        Freq       Total Category  logprob  loglift
31     section   64.000000   64.000000  Default  30.0000  30.0000
23        fund   74.000000   74.000000  Default  29.0000  29.0000
22   violation   93.000000   93.000000  Default  28.0000  28.0000
16     penalty  106.000000  106.000000  Default  27.0000  27.0000
365     market   44.000000   44.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
863    augment    1.267717    2.942303  Topic12  -4.6000   2.8213
541       wear    0.523613    1.258614  Topic12  -5.4842   2.7862
539      rider    0.523613    1.258614  Topic12  -5.4842   2.7862
536     helmet    0.523613    1.258614  Topic12  -5.4842   2.7862
534      crash    0.523613    1.258614  Topic12  -5.4842   2.7862

[456 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
283       8  0.811883  accreditation
80        4  0.982201            act
38        4  0.793144       activity
352       3  0.964907     additional
1138      7  0.859846       adjacent
...     ...       ...            ...
108       5  0.779218        welfare
919      11  0.793155           work
77        9  0.898335         worker
55        4  0.982115           year
79        7  0.840335           zone

[433 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 12, 10, 7, 1, 8, 5, 11, 9, 2, 3])